In [1]:
from transformers import TFAutoModel, TFBertForSequenceClassification, BertConfig
import numpy as np 

/Users/USER/Desktop/Side_project/transformer-classifier/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
bert_model = TFAutoModel.from_pretrained('BAAI/bge-base-en-v1.5')

2024-02-11 14:38:13.991461: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2024-02-11 14:38:13.991482: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-02-11 14:38:13.991487: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-02-11 14:38:13.991521: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-02-11 14:38:13.991539: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['embeddings.position_ids']
- This IS expected if you are initializing TFBertModel from a 

In [3]:
model_name = 'bge-base-en-v1.5'
Xids = np.load(f'./vectors/{model_name}/Xids.npy')
Xmasks = np.load(f'./vectors/{model_name}/Xmasks.npy')
labels = np.load(f'./vectors/{model_name}/labels.npy')

In [4]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [5]:
import tensorflow as tf

def create_model():
    input_ids = tf.keras.layers.Input(shape=(50, ), name='input_ids', dtype=tf.int32)
    mask = tf.keras.layers.Input(shape=(50, ), name='attention_mask', dtype=tf.int32)
    bert_model.trainable = False
    embeddings = bert_model.bert(input_ids=input_ids, attention_mask=mask)['pooler_output']
    # x = tf.keras.layers.GlobalMaxPooling1D()(embeddings)
    x = tf.keras.layers.Dense(50, activation='relu')(embeddings)
    x = tf.keras.layers.LayerNormalization()(x)
    x = tf.keras.layers.Reshape((-1, x.shape[-1]))(x)
    x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True))(x)
    x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=False))(x)
    x = tf.keras.layers.LayerNormalization()(x)
    x = tf.keras.layers.Dense(128, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    x = tf.keras.layers.Dense(32, activation='relu')(x)
    y = tf.keras.layers.Dense(5, activation='softmax', name='outputs')(x)

    model = tf.keras.Model(inputs=[input_ids, mask], outputs=y)
    optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=5e-05, epsilon=1e-08, decay=0.01, clipnorm=1.0)
    loss = tf.keras.losses.CategoricalCrossentropy()
    acc = tf.keras.metrics.CategoricalAccuracy()

    model.compile(optimizer=optimizer, loss=loss, metrics=[acc])

    return model

model = create_model()
model.load_weights("model/v2/v2")

In [8]:
model.save('./model/v2_saved.keras')

### predict test

In [10]:
import torch
from transformers import AutoTokenizer

In [16]:
import numpy as np

In [11]:
tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-base-en-v1.5")

In [43]:
tokens = tokenizer.encode_plus(text="It's like being moved after watching a movie.", max_length=50, 
                               truncation=True, padding='max_length', 
                               add_special_tokens=True, return_token_type_ids=False, 
                               return_attention_mask=True, return_tensors='tf')

In [44]:
results = model.predict({'input_ids': tokens['input_ids'], 'attention_mask': tokens['attention_mask']})

1/1 [==============================] - 0s 127ms/step


In [45]:
np.dot(np.array(results), np.transpose(np.array([1, 2, 3, 4, 5]))) / np.sum(results)

array([3.87283485])

In [46]:
results

array([[0.00530603, 0.05731955, 0.21556394, 0.50285447, 0.21895601]],
      dtype=float32)